In [1]:
import json
import random
from keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Conv2D
import numpy
from gensim.models.fasttext import load_facebook_model
from sklearn.model_selection import train_test_split
from features import Features

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import TweetTokenizer
from nltk import tokenize
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer



lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()
global comments
global tokens
global max_len
global labels
parents = []


trained_model = None
f = Features()
f.test()

Using TensorFlow backend.


features setup correctly


In [2]:
global fasttext_model

discourse = ['other', 'agreement', 'announcement', 'appreciation', 'humor', 'answer', 'elaboration', 'negativereaction',
             'question', 'disagreement']

In [3]:
def get_data(filename):
    load_json_data = []
    count = 0
    with open(filename) as jsonfile:
        for line in jsonfile:
            jline = json.loads(line)
            load_json_data.append(jline)
            count += 1
    return load_json_data

In [4]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [5]:
def process_body(load_data):
    global comments
    global tokens
    global max_len
    global labels
    comments = []
    labels = []
    count = 0
    for jline in load_data:
        for post in jline['posts']:
            try:
                features = []
                b = post['body']
                p = f.getParentBody(jline, post['id'])
                label = discourse.index(post['majority_type'])
                comments.append(b)
                labels.append(label)
                parents.append(p)
            except Exception as e:
                count += 1
    print(count)
    print(len(comments))
    print(len(parents))
    print(len(labels))
    return "Done"

In [6]:
global comments
global labels
json_data = get_data("coarse_discourse_dump_reddit.jsonlist")
print(process_body(json_data))
X = comments
Y = labels

14376
101981
101981
101981
Done


In [7]:
global comments
global labels
vectorizer = TfidfVectorizer(ngram_range = (1,3), min_df=(100/len(X))).fit(X)
tf_X = vectorizer.transform(X).toarray()
tf_parent = vectorizer.transform(parents).toarray()#//shape - (3,6)
tf_X = tf_X[:, None, :]

print(tf_X.shape)

# train(tf_X, labels)

(101981, 1, 7495)


In [ ]:
train(tf_X, labels)

In [8]:
def set_model(data):
    global trained_model
    input_shape = data[0].shape
    # After trying a bunch of different methods, this one worked the best
    t_model = tf.keras.Sequential([
        Bidirectional(LSTM(100, input_shape=input_shape)),
        # Bidirectional(LSTM(100)),
        # Bidirectional(LSTM(64)),
        #hub_layer,
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.25),
        Dense(32, activation='relu'),
        
        #tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    t_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['categorical_accuracy'])
    trained_model = t_model

def train(data, labels):
    global trained_model
    checkpoint_path = 'training_1/cp.ckpt'
    set_model(data)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
    trained_model.fit(data, to_categorical(labels), validation_split=0.1, epochs=100, batch_size=128, callbacks=[cp_callback])

In [9]:
def get_data(filename):
    load_json_data = []
    count = 0
    with open(filename) as jsonfile:
        for line in jsonfile:
            jline = json.loads(line)
            load_json_data.append(jline)
            count += 1
    return load_json_data

In [10]:
def process_data(load_data):
    global errors
    global f
    count = 0
    count_no_author = 0
    count_no_title = 0
    process_data_list = []
    process_label_list = []
    
    
    
    for jline in load_data:
#        author = None
#         if 'author' in jline['posts'][0]:
#             author = jline['posts'][0]['author']
        for post in jline['posts']:
            try:
                this_post = post['body']
                
                # Structure
                features = f.getStructureFeatures(jline, post['id'])
                # Content
                b = post['body']
                    
                # Author
                features.append(f.isSameAuthor(jline, post))
                
#                 if author == post['author']:
#                     features.append(numpy.full(300, 1.0))
#                 else:
#                     features.append(numpy.full(300, 0.0))
              
                
                
#                 if 'title' in jline:
#                     features.append(fasttext_Vec(jline['title']))
#                 else:
#                     features.append(numpy.zeros(300))
                    
                
                    
                    
                    
#                 # Community
#                 if 'subreddit' in jline:
#                     features.append(fasttext_Vec(jline['subreddit']))
#                 else:
#                     features.append(numpy.zeros(300))
                    
                # Thread
                features += f.thread_info(jline)
                
                
                parent_post = f.getParentBody(jline, post['id'])
                features.append(numpy.array(vectorizer.transform([parent_post]).toarray()[0]))
                features.append(numpy.array(vectorizer.transform([this_post]).toarray()[0]))
            
            
            
            
            
                feature_nparr = numpy.array(features)
                label = discourse.index(post['majority_type'])
                process_label_list.append([label])
                process_data_list.append(feature_nparr)
            except Exception as e:
                count += 1
    print("Total exception count: " + str(count))
    print("No authors: " + str(count_no_author))
    print("No titles: " + str(count_no_title))
    process_data_list = numpy.array(process_data_list)
    process_label_list = numpy.array(process_label_list)
    print(process_data_list.shape)
    print(process_label_list.shape)
    print("done processing")
    return process_data_list, process_label_list

In [11]:

json_data = get_data("coarse_discourse_dump_reddit.jsonlist")


In [12]:
data, labels = process_data(json_data)

Total exception count: 14718
No authors: 0
No titles: 0
(101639, 14)
(101639, 1)
done processing


In [17]:
test_vec = vectorizer.transform(["Hello! How are you doing?"]).toarray()
print(test_vec)

[[0. 0. 0. ... 0. 0. 0.]]


In [16]:
data.shape


(101639, 14)

In [18]:
new_data = numpy.array([numpy.concatenate((numpy.array([datum[0][0]]), numpy.array([datum[1][0]]), numpy.array([datum[2][0]]), numpy.array([datum[3][0]]), numpy.array([datum[4][0]]), numpy.array([datum[5][0]]), numpy.array([datum[6][0]]), numpy.array([datum[7][0]]), numpy.array([datum[8][0]]), numpy.array([datum[9][0]]), numpy.array([datum[10][0]]), numpy.array([datum[11][0]]), numpy.array(datum[12]), numpy.array(datum[13])), axis=None) for datum in data])

In [ ]:
print(numpy.array(new_data).shape)

In [18]:
labels

array([[2],
       [6],
       [6],
       ...,
       [5],
       [5],
       [6]])

In [14]:
new_data = new_data[:, None, :]
print(new_data.shape)

(101639, 1, 15002)


In [ ]:
new_data = [datam[0] for datam in new_data]

In [ ]:
train(new_data, labels)

Train on 91475 samples, validate on 10164 samples
Epoch 1/100
91008/91475 [============================>.] - ETA: 0s - loss: 1.6053 - categorical_accuracy: 0.4432
Epoch 00001: saving model to training_1/cp.ckpt
91475/91475 [==============================] - 11s 122us/sample - loss: 1.6052 - categorical_accuracy: 0.4433 - val_loss: 1.5363 - val_categorical_accuracy: 0.4507
Epoch 2/100
91136/91475 [============================>.] - ETA: 0s - loss: 1.4888 - categorical_accuracy: 0.4879
Epoch 00002: saving model to training_1/cp.ckpt
91475/91475 [==============================] - 9s 99us/sample - loss: 1.4883 - categorical_accuracy: 0.4881 - val_loss: 1.4266 - val_categorical_accuracy: 0.5009
Epoch 3/100
91392/91475 [============================>.] - ETA: 0s - loss: 1.4113 - categorical_accuracy: 0.5177
Epoch 00003: saving model to training_1/cp.ckpt
91475/91475 [==============================] - 9s 101us/sample - loss: 1.4113 - categorical_accuracy: 0.5177 - val_loss: 1.3550 - val_categor

In [31]:
print(data.shape)
print(data[0])

(101639, 14)
[array([0.]) array([2553.]) array([501.]) array([29.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([4.]) array([3.]) array([1.])
 array([1.])
 array([0.04366303, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])
 array([0.04366303, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])]


In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.externals import joblib
#
from sklearn.datasets import load_iris
#
from sklearn.model_selection import train_test_split


def run_KNN(data):
    X_train = data["x_tr"]
    X_test = data["x_te"]
    y_train = data["y_tr"]
    y_test = data["y_te"]

    # training the model on training set
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)

    # making predictions on the testing set
    y_pred_train = knn.predict(X_train)
    y_pred_test = knn.predict(X_test)

    # comparing actual response values (y_test) with predicted response values (y_pred)
    print("KNN Training Accuracy:", metrics.accuracy_score(y_train, y_pred_train))
    print("KNN Testing Accuracy:", metrics.accuracy_score(y_test, y_pred_test))

    # Example of making prediction for out of sample data
    # sample = [[3, 5, 4, 2], [2, 3, 5, 4]] # make sure it is proper size
    # preds = knn.predict(sample)
    # print("Predictions:", preds)

    # saving the model
    joblib.dump(knn, 'knn_model.pkl')

    # To load model use: knn = joblib.load('knn_model.pkl')


def run_SVM(data):
    X_train = data["x_tr"]
    X_test = data["x_te"]
    y_train = data["y_tr"]
    y_test = data["y_te"]

    # training the model on training set
    svm = SVC(gamma='auto', verbose=True, cache_size=101639)
    svm.fit(X_train, y_train)

    # making predictions on the testing set
    y_pred_train = svm.predict(X_train)
    y_pred_test = svm.predict(X_test)

    # comparing actual response values (y_test) with predicted response values (y_pred)
    print("SVM Training Accuracy:", metrics.accuracy_score(y_train, y_pred_train))
    print("SVM Testing Accuracy:", metrics.accuracy_score(y_test, y_pred_test))

    # Example of making prediction for out of sample data
    # sample = [[3, 5, 4, 2], [2, 3, 5, 4]] # make sure it is proper size
    # preds = svm.predict(sample)
    # print("Predictions:", preds)

    # saving the model
    joblib.dump(svm, 'svm_model.pkl')

    # To load model use: knn = joblib.load('svm_model.pkl')


def run_RandomForest(data):
    X_train = data["x_tr"]
    X_test = data["x_te"]
    y_train = data["y_tr"]
    y_test = data["y_te"]

    # training the model on training set
    rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    rf.fit(X_train, y_train)

    # making predictions on the testing set
    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)

    # comparing actual response values (y_test) with predicted response values (y_pred)
    print("RandomForest Training Accuracy:", metrics.accuracy_score(y_train, y_pred_train))
    print("RandomForest Testing Accuracy:", metrics.accuracy_score(y_test, y_pred_test))

    # Example of making prediction for out of sample data
    # sample = [[3, 5, 4, 2], [2, 3, 5, 4]] # make sure it is proper size
    # preds = rf.predict(sample)
    # print("Predictions:", preds)

    # saving the model
    joblib.dump(rf, 'rf_model.pkl')

    # To load model use: knn = joblib.load('rf_model.pkl')


def run_MLP(data):
    X_train = data["x_tr"]
    X_test = data["x_te"]
    y_train = data["y_tr"]
    y_test = data["y_te"]

    # training the model on training set
    mlp = MLPClassifier(solver='adam', hidden_layer_sizes=(128,))
    mlp.fit(X_train, y_train)

    # making predictions on the testing set
    y_pred_train = mlp.predict(X_train)
    y_pred_test = mlp.predict(X_test)

    # comparing actual response values (y_test) with predicted response values (y_pred)
    print("MLP Training Accuracy:", metrics.accuracy_score(y_train, y_pred_train))
    print("MLP Testing Accuracy:", metrics.accuracy_score(y_test, y_pred_test))

    # Example of making prediction for out of sample data
    # sample = [[3, 5, 4, 2], [2, 3, 5, 4]] # make sure it is proper size
    # preds = mlp.predict(sample)
    # print("Predictions:", preds)

    # saving the model
    joblib.dump(mlp, 'mlp_model.pkl')

    # To load model use: knn = joblib.load('mlp_model.pkl')


def run_LRG(data):
    X_train = data["x_tr"]
    X_test = data["x_te"]
    y_train = data["y_tr"]
    y_test = data["y_te"]

    # training the model on training set
    mlp = LogisticRegression()
    mlp.fit(X_train, y_train)

    # making predictions on the testing set
    y_pred_train = mlp.predict(X_train)
    y_pred_test = mlp.predict(X_test)

    # comparing actual response values (y_test) with predicted response values (y_pred)
    print("MLP Training Accuracy:", metrics.accuracy_score(y_train, y_pred_train))
    print("MLP Testing Accuracy:", metrics.accuracy_score(y_test, y_pred_test))

    # Example of making prediction for out of sample data
    # sample = [[3, 5, 4, 2], [2, 3, 5, 4]] # make sure it is proper size
    # preds = mlp.predict(sample)
    # print("Predictions:", preds)

    # saving the model
    joblib.dump(mlp, 'mlp_model.pkl')

    # To load model use: knn = joblib.load('mlp_model.pkl')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(new_data, labels, test_size=0.2, random_state=1)
data_dict = {"x_tr": X_train, "x_te": X_test, "y_tr": y_train, "y_te": y_test}

In [20]:
print()
print("#"*30)
print("\nLinear Regression:\n")

run_LRG(data_dict)

print()


##############################

Linear Regression:



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterati

MLP Training Accuracy: 0.6346865737723063
MLP Testing Accuracy: 0.5831365604092876



In [ ]:
print()
print("#"*30)
print("\nMULTILAYER PERCEPTRON:\n")

run_MLP(data_dict)

print()


##############################

MULTILAYER PERCEPTRON:



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
